In [1]:
# Set number of threads to use
import os
nthreads = 12
os.environ["MKL_NUM_THREADS"] = str(nthreads)
os.environ["NUMEXPR_NUM_THREADS"] = str(nthreads)
os.environ["OMP_NUM_THREADS"] = str(nthreads)

import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys
sys.path.insert(1,'../src/')

In [3]:
from supercellHelpers import *
from buildGRNHelpers import *
from MergeNetworksHelpers import *

In [4]:
#adata = sc.read('../data/microglia.h5ad')

In [5]:
#adata

In [6]:
#adata_merged = supercell_pipeline(adata,
#                                  ngenes=2000,
#                                  npcs=20,
#                                  ncell=500,
#                                  verbose=True)

In [7]:
adata_merged = sc.read('../temp_data/merged_adata.h5ad')

# limit genes to 2000 random genes
# this is to speed up computation for the example
np.random.seed(0)
adata_merged = adata_merged[:,np.random.choice(np.arange(adata_merged.shape[1]),
                                       2000,
                                       replace=False)]

In [8]:
adata_merged

View of AnnData object with n_obs × n_vars = 503 × 2000
    var: 'gene_ids', 'feature_types', 'genome', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'

In [ ]:
all_edges = []
for i in range(10):
    print(i)
    adata_saved = adata_merged.copy()
    
    # -1 all genes
    # 100 neighbors for each gene
    # 10 pcs
    # 0.7 subsample per run
    scing = grnBuilder(adata_merged, -1, 100, 10,0.7,
                      'test','test',12,int(2e9),True)
    scing.subsample_cells()

    scing.filter_genes()
    scing.filter_gene_connectivities()
    scing.build_grn()
    
    all_edges.append(scing.edges)

0
building local cluster
Loading data into memory...
Building dask graph...
Computing dask graph...
Closing client...
1
building local cluster
Loading data into memory...
Building dask graph...
Computing dask graph...


Exception in thread Profile:
Traceback (most recent call last):
  File "/u/home/r/rlittman/project-rlittman/miniconda3/envs/scing/lib/python3.10/site-packages/distributed/profile.py", line 115, in process
    d = state["children"][ident]
KeyError: 'indentcount_lines;/u/home/r/rlittman/project-rlittman/miniconda3/envs/scing/lib/python3.10/site-packages/scipy/_lib/doccer.py;177'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/u/home/r/rlittman/project-rlittman/miniconda3/envs/scing/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/u/home/r/rlittman/project-rlittman/miniconda3/envs/scing/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/u/home/r/rlittman/project-rlittman/miniconda3/envs/scing/lib/python3.10/site-packages/distributed/profile.py", line 274, in _watch
    process(frame, None, recent, omit=omit)
  File "/u/home/r/rlittman/pro

Closing client...
2
building local cluster
Loading data into memory...
Building dask graph...
Computing dask graph...


Exception in thread Profile:
Traceback (most recent call last):
  File "/u/home/r/rlittman/project-rlittman/miniconda3/envs/scing/lib/python3.10/site-packages/distributed/profile.py", line 115, in process
    d = state["children"][ident]
KeyError: 'indentcount_lines;/u/home/r/rlittman/project-rlittman/miniconda3/envs/scing/lib/python3.10/site-packages/scipy/_lib/doccer.py;177'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/u/home/r/rlittman/project-rlittman/miniconda3/envs/scing/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/u/home/r/rlittman/project-rlittman/miniconda3/envs/scing/lib/python3.10/threading.py", line 953, in run
Exception in thread Profile:
Traceback (most recent call last):
  File "/u/home/r/rlittman/project-rlittman/miniconda3/envs/scing/lib/python3.10/site-packages/distributed/profile.py", line 115, in process
    self._target(*self._args, **self._kwargs)
  File "/

Closing client...
3
building local cluster


In [ ]:
import pickle as pkl
with open('../temp_data/edge.pkl','wb') as f:
    pkl.dump(all_edges, f)
    #all_edges = pkl.load(f)

In [ ]:
merger = NetworkMerger(adata_merged,
                    all_edges,
                       0.2,
                    'test',
                    'test',
                    12,
                    int(2e9),
                    True)

In [ ]:
merger.preprocess_network_files()
merger.remove_reversed_edges()
merger.remove_cycles()
merger.get_triads()
merger.remove_redundant_edges()

In [ ]:
merger.edge_df.sort_values(by='importance',
                          ascending=False)

In [ ]:
all_edges